## Regresión Lineal y Polinómica para Precios de Calabazas - Lección 3
<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografía por Dasani Madipalli</figcaption>


#### Introducción

Hasta ahora has explorado qué es la regresión utilizando datos de muestra obtenidos del conjunto de datos sobre precios de calabazas que usaremos a lo largo de esta lección. También lo has visualizado usando `ggplot2`. 💪

Ahora estás listo para profundizar en la regresión para ML. En esta lección, aprenderás más sobre dos tipos de regresión: *regresión lineal básica* y *regresión polinómica*, junto con algunas matemáticas que sustentan estas técnicas.

> A lo largo de este currículo, asumimos un conocimiento mínimo de matemáticas y buscamos hacerlo accesible para estudiantes provenientes de otros campos, así que presta atención a notas, 🧮 llamados, diagramas y otras herramientas de aprendizaje para facilitar la comprensión.

#### Preparación

Como recordatorio, estás cargando estos datos para hacer preguntas sobre ellos.

-   ¿Cuándo es el mejor momento para comprar calabazas?

-   ¿Qué precio puedo esperar por una caja de calabazas miniatura?

-   ¿Debería comprarlas en cestas de medio bushel o en cajas de 1 1/9 bushel? Sigamos investigando estos datos.

En la lección anterior, creaste un `tibble` (una reinterpretación moderna del marco de datos) y lo llenaste con parte del conjunto de datos original, estandarizando los precios por bushel. Sin embargo, al hacer eso, solo pudiste recopilar alrededor de 400 puntos de datos y solo para los meses de otoño. ¿Quizás podamos obtener un poco más de detalle sobre la naturaleza de los datos limpiándolos más? Veremos... 🕵️‍♀️

Para esta tarea, necesitaremos los siguientes paquetes:

-   `tidyverse`: El [tidyverse](https://www.tidyverse.org/) es una [colección de paquetes de R](https://www.tidyverse.org/packages) diseñada para hacer la ciencia de datos más rápida, fácil y divertida.

-   `tidymodels`: El marco [tidymodels](https://www.tidymodels.org/) es una [colección de paquetes](https://www.tidymodels.org/packages/) para modelado y aprendizaje automático.

-   `janitor`: El paquete [janitor](https://github.com/sfirke/janitor) proporciona herramientas simples para examinar y limpiar datos sucios.

-   `corrplot`: El paquete [corrplot](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html) ofrece una herramienta visual exploratoria sobre matrices de correlación que admite el reordenamiento automático de variables para ayudar a detectar patrones ocultos entre variables.

Puedes instalarlos con:

`install.packages(c("tidyverse", "tidymodels", "janitor", "corrplot"))`

El siguiente script verifica si tienes los paquetes necesarios para completar este módulo y los instala por ti en caso de que falten.


In [ ]:
suppressWarnings(if (!require("pacman")) install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, corrplot)

Más adelante cargaremos estos paquetes increíbles y los haremos disponibles en nuestra sesión actual de R. (Esto es solo para ilustración, `pacman::p_load()` ya hizo eso por ti)

## 1. Una línea de regresión lineal

Como aprendiste en la Lección 1, el objetivo de un ejercicio de regresión lineal es poder trazar una *línea* *de* *mejor ajuste* para:

-   **Mostrar relaciones entre variables**. Mostrar la relación entre las variables.

-   **Hacer predicciones**. Hacer predicciones precisas sobre dónde caería un nuevo punto de datos en relación con esa línea.

Para dibujar este tipo de línea, usamos una técnica estadística llamada **Regresión por Mínimos Cuadrados**. El término `mínimos cuadrados` significa que todos los puntos de datos que rodean la línea de regresión se elevan al cuadrado y luego se suman. Idealmente, esa suma final es lo más pequeña posible, porque queremos un número bajo de errores, o `mínimos cuadrados`. Por lo tanto, la línea de mejor ajuste es la línea que nos da el valor más bajo para la suma de los errores al cuadrado, de ahí el nombre *regresión por mínimos cuadrados*.

Hacemos esto porque queremos modelar una línea que tenga la menor distancia acumulativa de todos nuestros puntos de datos. También elevamos los términos al cuadrado antes de sumarlos, ya que nos preocupa su magnitud más que su dirección.

> **🧮 Muéstrame las matemáticas**
>
> Esta línea, llamada la *línea de mejor ajuste*, puede expresarse mediante [una ecuación](https://es.wikipedia.org/wiki/Regresi%C3%B3n_lineal_simple):
>
>     Y = a + bX
>
> `X` es la '`variable explicativa` o `predictor`'. `Y` es la '`variable dependiente` o `resultado`'. La pendiente de la línea es `b` y `a` es la intersección con el eje Y, que se refiere al valor de `Y` cuando `X = 0`.
>

> ![](../../../../../../2-Regression/3-Linear/solution/images/slope.png "pendiente = $y/x$")
    Infografía por Jen Looper
>
> Primero, calcula la pendiente `b`.
>
> En otras palabras, y refiriéndonos a la pregunta original de los datos de las calabazas: "predecir el precio de una calabaza por bushel según el mes", `X` se referiría al precio y `Y` se referiría al mes de venta.
>
> ![](../../../../../../translated_images/calculation.989aa7822020d9d0ba9fc781f1ab5192f3421be86ebb88026528aef33c37b0d8.es.png)
    Infografía por Jen Looper
> 
> Calcula el valor de Y. Si estás pagando alrededor de \$4, ¡debe ser abril!
>
> Las matemáticas que calculan la línea deben demostrar la pendiente de la línea, que también depende de la intersección, o dónde se sitúa `Y` cuando `X = 0`.
>
> Puedes observar el método de cálculo para estos valores en el sitio web [Math is Fun](https://www.mathsisfun.com/data/least-squares-regression.html). También visita [este calculador de mínimos cuadrados](https://www.mathsisfun.com/data/least-squares-calculator.html) para ver cómo los valores de los números impactan la línea.

No es tan aterrador, ¿verdad? 🤓

#### Correlación

Otro término que debes entender es el **Coeficiente de Correlación** entre las variables X y Y dadas. Usando un diagrama de dispersión, puedes visualizar rápidamente este coeficiente. Un gráfico con puntos de datos dispersos en una línea ordenada tiene alta correlación, pero un gráfico con puntos de datos dispersos por todas partes entre X y Y tiene baja correlación.

Un buen modelo de regresión lineal será aquel que tenga un Coeficiente de Correlación alto (más cercano a 1 que a 0) utilizando el método de Regresión por Mínimos Cuadrados con una línea de regresión.


## **2. Un baile con los datos: creando un marco de datos que será utilizado para modelar**

<p >
   <img src="../../images/janitor.jpg"
   width="700"/>
   <figcaption>Obra de arte por @allison_horst</figcaption>


<!--¡Obra de arte por \@allison_horst](../../images/janitor.jpg){width="700"}-->


Carga las bibliotecas necesarias y el conjunto de datos. Convierte los datos a un marco de datos que contenga un subconjunto de los datos:

-   Obtén únicamente calabazas con precios por fanega.

-   Convierte la fecha a un mes.

-   Calcula el precio como un promedio entre los precios altos y bajos.

-   Convierte el precio para reflejar la cantidad por fanega.

> Cubrimos estos pasos en la [lección anterior](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/2-Data/solution/lesson_2-R.ipynb).


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)
library(lubridate)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n = 5)

En el espíritu de pura aventura, exploremos el [`paquete janitor`](../../../../../../2-Regression/3-Linear/solution/R/github.com/sfirke/janitor) que proporciona funciones simples para examinar y limpiar datos desordenados. Por ejemplo, echemos un vistazo a los nombres de las columnas de nuestros datos:


In [ ]:
# Return column names
pumpkins %>% 
  names()

🤔 Podemos hacerlo mejor. Convirtamos estos nombres de columna en `friendR` utilizando la convención [snake_case](https://es.wikipedia.org/wiki/Snake_case) con `janitor::clean_names`. Para obtener más información sobre esta función: `?clean_names`


In [ ]:
# Clean names to the snake_case convention
pumpkins <- pumpkins %>% 
  clean_names(case = "snake")

# Return column names
pumpkins %>% 
  names()

¡Mucho tidyR 🧹! Ahora, ¡un baile con los datos usando `dplyr` como en la lección anterior! 💃


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(variety, city_name, package, low_price, high_price, date)



# Extract the month from the dates to a new column
pumpkins <- pumpkins %>%
  mutate(date = mdy(date),
         month = month(date)) %>% 
  select(-date)



# Create a new column for average Price
pumpkins <- pumpkins %>% 
  mutate(price = (low_price + high_price)/2)


# Retain only pumpkins with the string "bushel"
new_pumpkins <- pumpkins %>% 
  filter(str_detect(string = package, pattern = "bushel"))


# Normalize the pricing so that you show the pricing per bushel, not per 1 1/9 or 1/2 bushel
new_pumpkins <- new_pumpkins %>% 
  mutate(price = case_when(
    str_detect(package, "1 1/9") ~ price/(1.1),
    str_detect(package, "1/2") ~ price*2,
    TRUE ~ price))

# Relocate column positions
new_pumpkins <- new_pumpkins %>% 
  relocate(month, .before = variety)


# Display the first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

¡Buen trabajo!👌 Ahora tienes un conjunto de datos limpio y ordenado sobre el cual puedes construir tu nuevo modelo de regresión.

¿Te apetece un diagrama de dispersión?


In [ ]:
# Set theme
theme_set(theme_light())

# Make a scatter plot of month and price
new_pumpkins %>% 
  ggplot(mapping = aes(x = month, y = price)) +
  geom_point(size = 1.6)


Un diagrama de dispersión nos recuerda que solo tenemos datos mensuales desde agosto hasta diciembre. Probablemente necesitemos más datos para poder sacar conclusiones de manera lineal.

Echemos un vistazo nuevamente a nuestros datos de modelado:


In [ ]:
# Display first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

¿Qué pasaría si quisiéramos predecir el `precio` de una calabaza basándonos en las columnas `ciudad` o `paquete`, que son de tipo carácter? O incluso más simple, ¿cómo podríamos encontrar la correlación (que requiere que ambos de sus valores sean numéricos) entre, por ejemplo, `paquete` y `precio`? 🤷🤷

Los modelos de aprendizaje automático funcionan mejor con características numéricas en lugar de valores de texto, por lo que generalmente necesitas convertir las características categóricas en representaciones numéricas.

Esto significa que debemos encontrar una manera de reformatear nuestros predictores para que sean más fáciles de usar por un modelo de manera efectiva, un proceso conocido como `ingeniería de características`.


## 3. Preprocesamiento de datos para modelado con recipes 👩‍🍳👨‍🍳

Las actividades que reformatean los valores de los predictores para que sean más fáciles de usar por un modelo se han denominado `ingeniería de características`.

Diferentes modelos tienen diferentes requisitos de preprocesamiento. Por ejemplo, los mínimos cuadrados requieren `codificar variables categóricas` como mes, variedad y city_name. Esto simplemente implica `traducir` una columna con `valores categóricos` en una o más `columnas numéricas` que reemplazan a la original.

Por ejemplo, supongamos que tus datos incluyen la siguiente característica categórica:

|  ciudad   |
|:---------:|
| Denver    |
| Nairobi   |
|  Tokio    |

Puedes aplicar *codificación ordinal* para sustituir un valor entero único para cada categoría, así:

| ciudad |
|:------:|
|   0    |
|   1    |
|   2    |

¡Y eso es lo que haremos con nuestros datos!

En esta sección, exploraremos otro increíble paquete de Tidymodels: [recipes](https://tidymodels.github.io/recipes/) - que está diseñado para ayudarte a preprocesar tus datos **antes** de entrenar tu modelo. En esencia, una receta es un objeto que define qué pasos deben aplicarse a un conjunto de datos para que esté listo para el modelado.

Ahora, vamos a crear una receta que prepare nuestros datos para el modelado sustituyendo un entero único para todas las observaciones en las columnas de predictores:


In [ ]:
# Specify a recipe
pumpkins_recipe <- recipe(price ~ ., data = new_pumpkins) %>% 
  step_integer(all_predictors(), zero_based = TRUE)


# Print out the recipe
pumpkins_recipe

¡Increíble! 👏 ¡Acabamos de crear nuestra primera receta que especifica un resultado (precio) y sus correspondientes predictores, y que todas las columnas de predictores deben codificarse en un conjunto de enteros 🙌! Vamos a desglosarlo rápidamente:

-   La llamada a `recipe()` con una fórmula le indica a la receta los *roles* de las variables utilizando los datos de `new_pumpkins` como referencia. Por ejemplo, la columna `price` ha sido asignada como un rol de `outcome`, mientras que el resto de las columnas han sido asignadas como rol de `predictor`.

-   `step_integer(all_predictors(), zero_based = TRUE)` especifica que todos los predictores deben convertirse en un conjunto de enteros, comenzando la numeración desde 0.

Estamos seguros de que podrías estar pensando algo como: "¡Esto es genial! Pero, ¿qué pasa si necesito confirmar que las recetas están haciendo exactamente lo que espero que hagan? 🤔"

¡Esa es una excelente pregunta! Verás, una vez que defines tu receta, puedes estimar los parámetros necesarios para preprocesar realmente los datos y luego extraer los datos procesados. Normalmente no necesitas hacer esto cuando usas Tidymodels (veremos la convención normal en un momento -> `workflows`), pero puede ser útil cuando quieres hacer algún tipo de verificación para confirmar que las recetas están funcionando como esperas.

Para eso, necesitarás dos verbos más: `prep()` y `bake()`, y como siempre, nuestros pequeños amigos de R creados por [`Allison Horst`](https://github.com/allisonhorst/stats-illustrations) te ayudarán a entender esto mejor.

<p >
   <img src="../../images/recipes.png"
   width="550"/>
   <figcaption>Ilustración por @allison_horst</figcaption>


<!--![Ilustración por \@allison_horst](../../../../../../translated_images/recipes.9ad10d8a4056bf89413fc33644924e0bd29d7c12fb2154e03a1ca3d2d6ea9323.es.png){width="550"}-->


[`prep()`](https://recipes.tidymodels.org/reference/prep.html): estima los parámetros necesarios a partir de un conjunto de entrenamiento que luego pueden aplicarse a otros conjuntos de datos. Por ejemplo, para una columna predictora dada, qué observación será asignada como entero 0, 1, 2, etc.

[`bake()`](https://recipes.tidymodels.org/reference/bake.html): toma una receta preparada y aplica las operaciones a cualquier conjunto de datos.

Dicho esto, preparemos y apliquemos nuestras recetas para confirmar realmente que, en el fondo, las columnas predictoras serán primero codificadas antes de ajustar un modelo.


In [ ]:
# Prep the recipe
pumpkins_prep <- prep(pumpkins_recipe)

# Bake the recipe to extract a preprocessed new_pumpkins data
baked_pumpkins <- bake(pumpkins_prep, new_data = NULL)

# Print out the baked data set
baked_pumpkins %>% 
  slice_head(n = 10)

¡Woo-hoo! 🥳 Los datos procesados `baked_pumpkins` tienen todos sus predictores codificados, confirmando que, efectivamente, los pasos de preprocesamiento definidos como nuestra receta funcionarán como se espera. Esto hace que sea más difícil de leer para ti, pero mucho más comprensible para Tidymodels. Tómate un momento para descubrir qué observación ha sido asignada a un entero correspondiente.

También vale la pena mencionar que `baked_pumpkins` es un marco de datos sobre el cual podemos realizar cálculos.

Por ejemplo, intentemos encontrar una buena correlación entre dos puntos de tus datos para potencialmente construir un buen modelo predictivo. Usaremos la función `cor()` para hacerlo. Escribe `?cor()` para obtener más información sobre la función.


In [ ]:
# Find the correlation between the city_name and the price
cor(baked_pumpkins$city_name, baked_pumpkins$price)

# Find the correlation between the package and the price
cor(baked_pumpkins$package, baked_pumpkins$price)


Resulta que solo hay una correlación débil entre la Ciudad y el Precio. Sin embargo, hay una mejor correlación entre el Paquete y su Precio. Tiene sentido, ¿verdad? Normalmente, cuanto más grande es la caja de productos, mayor es el precio.

Ya que estamos, intentemos también visualizar una matriz de correlación de todas las columnas utilizando el paquete `corrplot`.


In [ ]:
# Load the corrplot package
library(corrplot)

# Obtain correlation matrix
corr_mat <- cor(baked_pumpkins %>% 
                  # Drop columns that are not really informative
                  select(-c(low_price, high_price)))

# Make a correlation plot between the variables
corrplot(corr_mat, method = "shade", shade.col = NA, tl.col = "black", tl.srt = 45, addCoef.col = "black", cl.pos = "n", order = "original")

🤩🤩 Mucho mejor.

Una buena pregunta para hacer ahora con estos datos sería: '`¿Qué precio puedo esperar de un paquete de calabazas dado?`' ¡Vamos a ello!

> Nota: Cuando **`bake()`** la receta preparada **`pumpkins_prep`** con **`new_data = NULL`**, extraes los datos de entrenamiento procesados (es decir, codificados). Si tuvieras otro conjunto de datos, por ejemplo, un conjunto de prueba, y quisieras ver cómo una receta lo preprocesaría, simplemente hornearías **`pumpkins_prep`** con **`new_data = test_set`**

## 4. Construir un modelo de regresión lineal

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografía por Dasani Madipalli</figcaption>


<!--![Infografía por Dasani Madipalli](../../../../../../translated_images/linear-polynomial.5523c7cb6576ccab0fecbd0e3505986eb2d191d9378e785f82befcf3a578a6e7.es.png){width="800"}-->


Ahora que hemos creado una receta y confirmado que los datos se preprocesarán adecuadamente, construyamos un modelo de regresión para responder a la pregunta: `¿Qué precio puedo esperar de un paquete de calabaza dado?`

#### Entrenar un modelo de regresión lineal utilizando el conjunto de entrenamiento

Como probablemente ya te habrás dado cuenta, la columna *price* es la variable `resultado`, mientras que la columna *package* es la variable `predictora`.

Para hacer esto, primero dividiremos los datos de manera que el 80% se destine al conjunto de entrenamiento y el 20% al conjunto de prueba. Luego definiremos una receta que codifique la columna predictora en un conjunto de enteros, y después construiremos una especificación del modelo. No prepararemos ni aplicaremos nuestra receta, ya que sabemos que preprocesará los datos como se espera.


In [ ]:
set.seed(2056)
# Split the data into training and test sets
pumpkins_split <- new_pumpkins %>% 
  initial_split(prop = 0.8)


# Extract training and test data
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)



# Create a recipe for preprocessing the data
lm_pumpkins_recipe <- recipe(price ~ package, data = pumpkins_train) %>% 
  step_integer(all_predictors(), zero_based = TRUE)



# Create a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")

¡Buen trabajo! Ahora que tenemos una receta y una especificación del modelo, necesitamos encontrar una forma de combinarlas en un objeto que primero preprocese los datos (prep+bake detrás de escena), ajuste el modelo con los datos preprocesados y también permita posibles actividades de postprocesamiento. ¡Qué tranquilidad, ¿verdad?! 🤩

En Tidymodels, este práctico objeto se llama [`workflow`](https://workflows.tidymodels.org/) y contiene convenientemente tus componentes de modelado. Esto es lo que llamaríamos *pipelines* en *Python*.

¡Así que vamos a agrupar todo en un workflow! 📦


In [ ]:
# Hold modelling components in a workflow
lm_wf <- workflow() %>% 
  add_recipe(lm_pumpkins_recipe) %>% 
  add_model(lm_spec)

# Print out the workflow
lm_wf

👌 Además, un flujo de trabajo puede ajustarse/entrenarse de manera muy similar a como se entrena un modelo.


In [ ]:
# Train the model
lm_wf_fit <- lm_wf %>% 
  fit(data = pumpkins_train)

# Print the model coefficients learned 
lm_wf_fit

A partir de la salida del modelo, podemos observar los coeficientes aprendidos durante el entrenamiento. Representan los coeficientes de la línea de mejor ajuste que nos da el menor error general entre la variable real y la predicha.

#### Evaluar el rendimiento del modelo usando el conjunto de prueba

¡Es hora de ver cómo se desempeñó el modelo 📏! ¿Cómo hacemos esto?

Ahora que hemos entrenado el modelo, podemos usarlo para hacer predicciones para el conjunto_de_prueba utilizando `parsnip::predict()`. Luego podemos comparar estas predicciones con los valores reales de las etiquetas para evaluar qué tan bien (¡o no!) está funcionando el modelo.

Comencemos haciendo predicciones para el conjunto de prueba y luego unamos las columnas al conjunto de prueba.


In [ ]:
# Make predictions for the test set
predictions <- lm_wf_fit %>% 
  predict(new_data = pumpkins_test)


# Bind predictions to the test set
lm_results <- pumpkins_test %>% 
  select(c(package, price)) %>% 
  bind_cols(predictions)


# Print the first ten rows of the tibble
lm_results %>% 
  slice_head(n = 10)

¡Sí, acabas de entrenar un modelo y usarlo para hacer predicciones!🔮 ¿Es bueno? Vamos a evaluar el rendimiento del modelo.

En Tidymodels, hacemos esto utilizando `yardstick::metrics()`! Para la regresión lineal, enfoquémonos en las siguientes métricas:

-   `Root Mean Square Error (RMSE)`: La raíz cuadrada del [MSE](https://en.wikipedia.org/wiki/Mean_squared_error). Esto proporciona una métrica absoluta en la misma unidad que la etiqueta (en este caso, el precio de una calabaza). Cuanto más pequeño sea el valor, mejor será el modelo (en términos simples, representa el precio promedio por el cual las predicciones están equivocadas).

-   `Coefficient of Determination (usualmente conocido como R-squared o R2)`: Una métrica relativa en la que cuanto mayor sea el valor, mejor será el ajuste del modelo. En esencia, esta métrica representa cuánto de la variación entre los valores predichos y reales de la etiqueta el modelo es capaz de explicar.


In [ ]:
# Evaluate performance of linear regression
metrics(data = lm_results,
        truth = price,
        estimate = .pred)

Ahí va el rendimiento del modelo. Veamos si podemos obtener una mejor indicación visualizando un gráfico de dispersión del paquete y el precio, y luego usar las predicciones realizadas para superponer una línea de mejor ajuste.

Esto significa que tendremos que preparar y procesar el conjunto de prueba para codificar la columna de paquete y luego vincular esto a las predicciones realizadas por nuestro modelo.


In [ ]:
# Encode package column
package_encode <- lm_pumpkins_recipe %>% 
  prep() %>% 
  bake(new_data = pumpkins_test) %>% 
  select(package)


# Bind encoded package column to the results
lm_results <- lm_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
lm_results %>% 
  slice_head(n = 5)


# Make a scatter plot
lm_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "orange", size = 1.2) +
  xlab("package")
  


¡Genial! Como puedes ver, el modelo de regresión lineal no generaliza muy bien la relación entre un paquete y su precio correspondiente.

🎃 ¡Felicidades! Acabas de crear un modelo que puede ayudar a predecir el precio de algunas variedades de calabazas. Tu huerto de calabazas para las fiestas será hermoso. ¡Pero probablemente puedas crear un modelo mejor!

## 5. Construir un modelo de regresión polinómica

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografía por Dasani Madipalli</figcaption>


<!--![Infografía por Dasani Madipalli](../../../../../../translated_images/linear-polynomial.5523c7cb6576ccab0fecbd0e3505986eb2d191d9378e785f82befcf3a578a6e7.es.png){width="800"}-->


A veces nuestros datos pueden no tener una relación lineal, pero aún queremos predecir un resultado. La regresión polinómica puede ayudarnos a hacer predicciones para relaciones no lineales más complejas.

Tomemos, por ejemplo, la relación entre el paquete y el precio en nuestro conjunto de datos de calabazas. Aunque a veces hay una relación lineal entre las variables - cuanto mayor es la calabaza en volumen, mayor es el precio - en ocasiones estas relaciones no pueden representarse como un plano o una línea recta.

> ✅ Aquí tienes [algunos ejemplos adicionales](https://online.stat.psu.edu/stat501/lesson/9/9.8) de datos que podrían usar regresión polinómica
>
> Observa nuevamente la relación entre la variedad y el precio en el gráfico anterior. ¿Parece que este diagrama de dispersión debería analizarse necesariamente con una línea recta? Quizás no. En este caso, puedes intentar usar regresión polinómica.
>
> ✅ Los polinomios son expresiones matemáticas que pueden consistir en una o más variables y coeficientes

#### Entrenar un modelo de regresión polinómica usando el conjunto de entrenamiento

La regresión polinómica crea una *línea curva* para ajustar mejor los datos no lineales.

Veamos si un modelo polinómico puede desempeñarse mejor en la predicción. Seguiremos un procedimiento algo similar al que hicimos antes:

-   Crear una receta que especifique los pasos de preprocesamiento que deben realizarse en nuestros datos para prepararlos para el modelado, es decir: codificar predictores y calcular polinomios de grado *n*

-   Construir una especificación del modelo

-   Combinar la receta y la especificación del modelo en un flujo de trabajo

-   Crear un modelo ajustando el flujo de trabajo

-   Evaluar qué tan bien se desempeña el modelo con los datos de prueba

¡Vamos a ello!


In [ ]:
# Specify a recipe
poly_pumpkins_recipe <-
  recipe(price ~ package, data = pumpkins_train) %>%
  step_integer(all_predictors(), zero_based = TRUE) %>% 
  step_poly(all_predictors(), degree = 4)


# Create a model specification
poly_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")


# Bundle recipe and model spec into a workflow
poly_wf <- workflow() %>% 
  add_recipe(poly_pumpkins_recipe) %>% 
  add_model(poly_spec)


# Create a model
poly_wf_fit <- poly_wf %>% 
  fit(data = pumpkins_train)


# Print learned model coefficients
poly_wf_fit

  

#### Evaluar el rendimiento del modelo

👏👏 ¡Has creado un modelo polinómico! ¡Hagamos predicciones en el conjunto de prueba!


In [ ]:
# Make price predictions on test data
poly_results <- poly_wf_fit %>% predict(new_data = pumpkins_test) %>% 
  bind_cols(pumpkins_test %>% select(c(package, price))) %>% 
  relocate(.pred, .after = last_col())


# Print the results
poly_results %>% 
  slice_head(n = 10)

Woo-hoo, evaluemos cómo se desempeñó el modelo en el test_set usando `yardstick::metrics()`.


In [ ]:
metrics(data = poly_results, truth = price, estimate = .pred)

🤩🤩 Mucho mejor rendimiento.

El `rmse` disminuyó de aproximadamente 7 a aproximadamente 3, lo que indica una reducción del error entre el precio real y el precio predicho. Puedes *interpretar libremente* esto como que, en promedio, las predicciones incorrectas tienen un error de alrededor de \$3. El `rsq` aumentó de aproximadamente 0.4 a 0.8.

Todos estos indicadores muestran que el modelo polinómico funciona mucho mejor que el modelo lineal. ¡Buen trabajo!

¡Veamos si podemos visualizar esto!


In [ ]:
# Bind encoded package column to the results
poly_results <- poly_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
poly_results %>% 
  slice_head(n = 5)


# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "midnightblue", size = 1.2) +
  xlab("package")


¡Puedes ver una línea curva que se ajusta mejor a tus datos! 🤩

¡Puedes hacer esto aún más suave pasando una fórmula polinómica a `geom_smooth` así:


In [ ]:
# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_smooth(method = lm, formula = y ~ poly(x, degree = 4), color = "midnightblue", size = 1.2, se = FALSE) +
  xlab("package")

Muy parecido a una curva suave. 🤩

Aquí tienes cómo hacer una nueva predicción:


In [ ]:
# Make a hypothetical data frame
hypo_tibble <- tibble(package = "bushel baskets")

# Make predictions using linear model
lm_pred <- lm_wf_fit %>% predict(new_data = hypo_tibble)

# Make predictions using polynomial model
poly_pred <- poly_wf_fit %>% predict(new_data = hypo_tibble)

# Return predictions in a list
list("linear model prediction" = lm_pred, 
     "polynomial model prediction" = poly_pred)


El modelo `polynomial` tiene sentido, dado los gráficos de dispersión de `price` y `package`. ¡Y, si este modelo es mejor que el anterior al observar los mismos datos, necesitas presupuestar para estas calabazas más caras!

🏆 ¡Bien hecho! Creaste dos modelos de regresión en una sola lección. En la sección final sobre regresión, aprenderás sobre la regresión logística para determinar categorías.

## **🚀Desafío**

Prueba varias variables diferentes en este notebook para ver cómo la correlación corresponde a la precisión del modelo.

## [**Cuestionario posterior a la lección**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/14/)

## **Revisión y Autoestudio**

En esta lección aprendimos sobre la Regresión Lineal. Existen otros tipos importantes de Regresión. Lee sobre las técnicas Stepwise, Ridge, Lasso y Elasticnet. Un buen curso para aprender más es el [curso de Stanford sobre Aprendizaje Estadístico](https://online.stanford.edu/courses/sohs-ystatslearning-statistical-learning).

Si quieres aprender más sobre cómo usar el increíble marco de trabajo Tidymodels, consulta los siguientes recursos:

-   Sitio web de Tidymodels: [Comienza con Tidymodels](https://www.tidymodels.org/start/)

-   Max Kuhn y Julia Silge, [*Tidy Modeling with R*](https://www.tmwr.org/)*.*

###### **AGRADECIMIENTOS A:**

[Allison Horst](https://twitter.com/allison_horst?lang=es) por crear las increíbles ilustraciones que hacen que R sea más accesible y atractivo. Encuentra más ilustraciones en su [galería](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).



---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Si bien nos esforzamos por lograr precisión, tenga en cuenta que las traducciones automáticas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse como la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
